In [4]:
# Libraries 
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F


In [ ]:
class NTXentLoss(nn.Module):
    def __init__(self, temperature):
        super().__init__()
        self.criterion = nn.CrossEntropyLoss(reduction = 'sum') #reduction sum means output will be summed
        self.similiratiy = nn.CosineSimilarity(dim=1) #cossine similarity
        self.temperature = temperature #scaling factor

        @staticmethod # doesn't depent upon the instance of the class means doesn't require self or cls
        def mask_correlated_samples(batch_size):
            n = 2*batch_size # each samples have two augmentations in contrastive learning (original + augmented)
            mask = torch.ones((n, n), dtype= bool) # created a nXn matrix with all true values
            mask = mask.fill_diagonal_(0) # filled diagonal elements with zero = False; so that samples shoudn't be campare with it self

            # masking the elements
            for i in range(batch_size): # ensuring that correlated pairs are excluded from neagative samples
                mask[i, batch_size+i] = 0
                mask[batch_size+i, i] = 0
            return mask
        
        def forward(self, z_i, z_j):
            batch_size = z_j.shape[0]
            n = 2*batch_size
            z = torch.cat((z_i, z_j), dim = 1) # number of rows remains same but columns increased
            z = F.normalize(z, dim=1) # rows wise noramlization // it's common in contrastive learning

            mask = self.mask_correlated_samples(batch_size)
            simi_larity = self.similiratiy(z.unsqueeze(1), z.unsqueeze(0))

            simi_larity_i_j = torch.diag(simi_larity, batch_size)
            simi_larity_j_i = torch.diag(simi_larity, batch_size)

            # number of positive samples and negative samples
            positive_samples = torch.cat((simi_larity_i_j, simi_larity_j_i), dim=0).reshape(n,1)
            negative_samples = simi_larity[mask].reshape(n, -1) # n rows and self decided columns

            labels = torch.from_numpy(np.array([0]*n)).reshape(-1).to(positive_samples.device).long() #flatten the output and sending it to the GPU converting it in the GPU supporting format; and zero represents correct class
            logits = torch.cat((positive_samples, negative_samples), dim=1) / self.temperature #similarity score

            loss = self.criterion(logits, labels)
            loss /= n

            return loss, (labels, logits)

In [7]:
torch.ones((4, 4), dtype= bool).fill_diagonal_(0)

tensor([[False,  True,  True,  True],
        [ True, False,  True,  True],
        [ True,  True, False,  True],
        [ True,  True,  True, False]])

## Loss Function
---
### NTXent Loss

## Pre-trained Modality Specific Backbone